In [1]:
from pathlib import Path

import pandas as pd

In [56]:
CLASSES = ['Healthy', 'OCDegen']
input_stages = {'0': 'training', '1': 'validation_nodiag', '2': 'validation_noxai'}
BASEPATH = Path('/Users/djohnson/sciebo/Human-Centric XAI AG/1. Projects/human-ai collab/1. Research/ijcai25/data')

high_stakes_pth = BASEPATH / 'example_validation_study'
low_stakes_pth = BASEPATH / 'example_validation_study_ls'

limesurvey_csv = BASEPATH / 'limesurvey_results_preprocessed_full.csv'
limesurvey_anon_csv = BASEPATH / 'limesurvey_results_anon.csv'

In [57]:
df_lime = pd.read_csv(limesurvey_csv, sep=';')
df_lime['Prolific ID'] = df_lime['Prolific ID'].astype(str)

In [58]:
# get unique prolific IDs and map to new ids for anonymization
pids = df_lime['Prolific ID'].unique()
id_map = {pid: f'study1-{new_id:02d}' for new_id, pid in enumerate(pids, start=1)}
len(pids)

60

In [59]:
# update dataframees with ids
df_lime['Prolific ID'] = df_lime['Prolific ID'].map(id_map)
df_lime.head(3)

,Prolific ID,Stakes,Response ID,Date submitted,Date started,Date last action,Study ID,Age,Gender,Education,...,Group time: Dia...4,Group time: Dia...5,Group time: Tut..,Group time: Pha..,Group time: Pha...1,Group time: Dia...6,Group time: Dia...7,Group time: Deb..,Group time: Tha..,status
0,study1-01,HIGH,60,2025-01-21 18:29:10,2025-01-21 18:09:11,2025-01-21 18:29:10,678f63e240fa98f8b965a7fc,22,Male,"Secondary education (e.g., High school diploma...",...,"60,32","230,06","9,2","138,32","157,54","9,05","260,64","14,05","2,06",keep
1,study1-02,LOW,34,2025-01-19 17:35:54,2025-01-19 17:14:46,2025-01-19 17:35:54,678c1954d2364179b6a7a7db,46,Female,"Doctoral degree (e.g., PhD, EdD, or equivalent)",...,"5,39","250,27","22,11","252,61","282,11","4,56","175,69","19,97","3,24",keep
2,study1-03,LOW,44,2025-01-21 16:15:06,2025-01-21 15:53:03,2025-01-21 16:15:06,678f6356b3ef4faa47a92ef4,43,Female,"Bachelor’s degree (e.g., BA, BSc, or equivalent)",...,"27,9","103,58","7,99","143,83","156,37","4,23","143,75","29,23","2,66",keep


In [ ]:
# save mapped dataframe to disk
df_lime.to_csv(limesurvey_anon_csv, sep=';', index=False)

In [ ]:
# rename results files with mapped ids
stakes_paths = [high_stakes_pth, low_stakes_pth]
for p in stakes_paths:
  for stage, name in input_stages.items():
    print(BASEPATH / p / name)
    print(len(list((BASEPATH / p / name).rglob('*.csv'))))
    for file in list((BASEPATH / p / name).rglob('*.csv')):
      stem = file.stem.split('_')
      pid = stem[-1]
      if pid not in id_map:
        print(f'No mapping for {pid}')
      else:
        stem[-1] = id_map[pid]
      stem = '_'.join(stem)
      new_file = file.with_name(stem + '.csv')
      file.rename(new_file)

In [ ]:
# get new names for rejected files
dj_filters = ['123451234512345123451234', '660ff395894d843a4c0c025b', 
              '678e7b0d4134f95ff9fbbf31', '6776c22a53ab42c3b0f49c2e',
              '669bd6a59ca25181f5222be9'
              ]

rejected = ['678b4ff772b1e5c63e30b608', '669a764c2a59d2c81e8ff68e', '668158082c54df1452c5875d', 
            '615a028ae7c469f84dcd7fcd', '5da83b70f3556d0014a1e273', '5ae96d1878b0060001e69b8a',
            '678e7b0d4134f95ff9fbbf31']

for f in filters:
  print(id_map[f])